In [49]:
import pandas as pd
import torch
import sys
import cv2
import numpy as np
from tqdm import tqdm
sys.path.append('thirdparty/ABAW2021')

In [2]:
# sys.path.remove('/mnt/DATA2/congvm/Workspace/ABAW2')
from utils.datamodule import get_transform, AffWildDataModule

In [47]:
data_module = AffWildDataModule(
    data_dir="/mnt/DATA1/hung/ABAW/data",
    backbone_name="vggresnet50",
    mode="static",
    batch_size=1,
)
data_module.setup(stage="fit")
val_dataloader = data_module.val_dataloader()


> Get ImageNet transform funcs


100%|██████████| 70/70 [00:00<00:00, 144.85it/s]


In [4]:
for batch in val_dataloader:
    break
print(batch['img_arr'].shape)
print(batch['img_path'])

torch.Size([1, 3, 112, 112])
['/mnt/DATA1/hung/ABAW/data/cropped_aligned/6-30-1920x1080_right/00001.jpg']


In [5]:
# Check transform
from torchvision import transforms
test_transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize(size=(112, 112)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

In [6]:
img_arr = cv2.imread(batch['img_path'][0])[..., ::-1]
img_arr = test_transform(img_arr)

In [7]:
batch['img_arr'][0].numpy()

array([[[-1.5356624 , -1.6041614 , -1.5870366 , ..., -1.2787911 ,
         -1.2102921 , -1.1589178 ],
        [-1.5699118 , -1.6384109 , -1.6555357 , ..., -1.3301654 ,
         -1.2616663 , -1.2787911 ],
        [-1.6212862 , -1.6555357 , -1.6726604 , ..., -1.3986644 ,
         -1.4157891 , -1.4500387 ],
        ...,
        [-1.3815396 , -1.3815396 , -1.3815396 , ..., -1.6897851 ,
         -1.6384109 , -1.5870366 ],
        [-1.4157891 , -1.3986644 , -1.3986644 , ..., -1.7240347 ,
         -1.6555357 , -1.5699118 ],
        [-1.4329139 , -1.4329139 , -1.4157891 , ..., -1.7411594 ,
         -1.6555357 , -1.5527872 ]],

       [[-1.5280112 , -1.5630252 , -1.5630252 , ..., -1.317927  ,
         -1.247899  , -1.1953781 ],
        [-1.5630252 , -1.6155462 , -1.5805322 , ..., -1.317927  ,
         -1.30042   , -1.265406  ],
        [-1.5980392 , -1.6330532 , -1.5980392 , ..., -1.370448  ,
         -1.405462  , -1.4229691 ],
        ...,
        [-0.93277305, -0.93277305, -0.93277305, ..., -

In [8]:
img_arr.numpy()

array([[[-1.5356624 , -1.6041614 , -1.5870366 , ..., -1.2787911 ,
         -1.2102921 , -1.1589178 ],
        [-1.5699118 , -1.6384109 , -1.6555357 , ..., -1.3301654 ,
         -1.2616663 , -1.2787911 ],
        [-1.6212862 , -1.6555357 , -1.6726604 , ..., -1.3986644 ,
         -1.4157891 , -1.4500387 ],
        ...,
        [-1.3815396 , -1.3815396 , -1.3815396 , ..., -1.6897851 ,
         -1.6384109 , -1.5870366 ],
        [-1.4157891 , -1.3986644 , -1.3986644 , ..., -1.7240347 ,
         -1.6555357 , -1.5699118 ],
        [-1.4329139 , -1.4329139 , -1.4157891 , ..., -1.7411594 ,
         -1.6555357 , -1.5527872 ]],

       [[-1.5280112 , -1.5630252 , -1.5630252 , ..., -1.317927  ,
         -1.247899  , -1.1953781 ],
        [-1.5630252 , -1.6155462 , -1.5805322 , ..., -1.317927  ,
         -1.30042   , -1.265406  ],
        [-1.5980392 , -1.6330532 , -1.5980392 , ..., -1.370448  ,
         -1.405462  , -1.4229691 ],
        ...,
        [-0.93277305, -0.93277305, -0.93277305, ..., -

In [9]:
(batch['img_arr'][0].numpy() - img_arr.numpy()).sum()

0.0

In [10]:
np.array_equal(batch['img_arr'][0].numpy(), img_arr.numpy())

True

In [11]:
PATH = "/mnt/DATA1/hung/ABAW/src/weight/multitask_best_ex_3.pth"
model = torch.load(PATH)
_ = model.eval()
# state_dict = ckpt.state_dict()
# state_dict_ = state_dict.copy()
# for k, v in state_dict.items():
#     state_dict_[k.replace('module.', '')] = v

In [12]:
val_dataloader = iter(val_dataloader)

In [46]:
batch = next(val_dataloader)
y_pred_exp, y_pred_au = model(batch['img_arr'])
y_pred_au = y_pred_au.sigmoid()
y_pred_au[y_pred_au >= 0.5] = 1.0
y_pred_au[y_pred_au < 0.5] = 0.0

y_pred_exp = torch.softmax(y_pred_exp, dim=1)
y_pred_exp_prob, y_pred_exp = torch.max(y_pred_exp, dim=1)

y_pred_exp_prob = y_pred_exp_prob.detach().cpu().numpy()
y_pred_exp = y_pred_exp.detach().cpu().numpy()
print(y_pred_exp_prob, y_pred_exp)
print(batch['labels'])

[0.8466097] [4]
tensor([4])


In [51]:
data_module = AffWildDataModule(
    data_dir="/mnt/DATA1/hung/ABAW/data",
    backbone_name="vggresnet50",
    mode="static",
    batch_size=32,
)
data_module.setup(stage="fit")
val_dataloader = data_module.val_dataloader()

_ = model.to("cuda:0")


> Get ImageNet transform funcs


100%|██████████| 70/70 [00:00<00:00, 143.92it/s]


In [54]:
# batch = next(val_dataloader)
exp_results = []
exp_labels = []
for batch in tqdm(val_dataloader):
    batch["img_arr"] = batch["img_arr"].to('cuda:0')
    with torch.no_grad():
        y_pred_exp, y_pred_au = model(batch["img_arr"])
    y_pred_au = y_pred_au.sigmoid()
    y_pred_au[y_pred_au >= 0.5] = 1.0
    y_pred_au[y_pred_au < 0.5] = 0.0

    y_pred_exp = torch.softmax(y_pred_exp, dim=1)
    y_pred_exp_prob, y_pred_exp = torch.max(y_pred_exp, dim=1)

    y_pred_exp_prob = y_pred_exp_prob.detach().cpu().numpy()
    y_pred_exp = y_pred_exp.detach().cpu().numpy()
    exp_results.extend(y_pred_exp)
    exp_labels.extend(batch["labels"])
    # print(y_pred_exp_prob, y_pred_exp)
    # print(batch["labels"])


100%|██████████| 7481/7481 [06:19<00:00, 19.69it/s]


In [58]:
exp_labels = [e.numpy() for e in exp_labels]

In [60]:
np.unique(exp_labels)

array([0, 1, 2, 3, 4, 5, 6])

In [62]:
from sklearn.metrics import f1_score
f1_score(exp_labels, exp_results, average='macro', labels=range(0, 7))

0.6503133229480594